[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Digital-AI-Finance/Digital-Finance-Introduction/blob/main/day_04/notebooks/NB09_AMM_Simulation.ipynb)

# NB09: Automated Market Maker (AMM) Simulation

**Topic:** 4.2 - Programmable Finance: Automated Market Makers

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand AMM Mechanics**: Learn how the constant product formula (x * y = k) enables decentralized trading
2. **Implement a Liquidity Pool**: Build a Uniswap-style AMM from scratch in Python
3. **Calculate Price Impact**: Understand how trade size affects execution price (slippage)
4. **Quantify Impermanent Loss**: Calculate and visualize the cost of providing liquidity
5. **Explore Arbitrage**: See how arbitrageurs keep AMM prices aligned with external markets
6. **Multi-hop Routing**: Implement token swaps through multiple pools

## Section 1: Setup

We'll use standard Python libraries for our AMM simulation. No blockchain needed - pure math!

In [ ]:
# Install required packages (if needed)
!pip install -q numpy pandas matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, List, Dict, Optional
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("Libraries loaded successfully!")
print("\nWe're ready to build an Automated Market Maker from scratch.")

## Section 2: Traditional Order Books vs AMMs

### How Traditional Exchanges Work

Traditional exchanges (NYSE, Binance, etc.) use **order books**:
- Buyers place **bid orders** ("I'll buy at this price")
- Sellers place **ask orders** ("I'll sell at this price")
- Trades execute when bid >= ask

**Problems with order books on blockchain:**
- Every order placement/cancellation costs gas
- Front-running is easy (miners see pending orders)
- Requires active market makers providing liquidity
- Low liquidity leads to wide spreads

### How AMMs Work

AMMs replace order books with **liquidity pools** and **mathematical formulas**:
- Liquidity providers deposit token pairs (e.g., ETH + USDC)
- A formula determines the exchange rate based on pool reserves
- Anyone can trade instantly against the pool
- No counterparty needed - trade with the contract itself!

**The key innovation:** Replace human market makers with math.

In [ ]:
# Visualize the difference between order books and AMMs

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Order Book visualization
ax1 = axes[0]
bid_prices = [99.5, 99.0, 98.5, 98.0, 97.5]
bid_sizes = [10, 25, 40, 30, 20]
ask_prices = [100.5, 101.0, 101.5, 102.0, 102.5]
ask_sizes = [15, 20, 35, 25, 15]

ax1.barh(bid_prices, bid_sizes, color='green', alpha=0.7, label='Bids (Buy Orders)')
ax1.barh(ask_prices, [-s for s in ask_sizes], color='red', alpha=0.7, label='Asks (Sell Orders)')
ax1.axhline(y=100, color='black', linestyle='--', label='Mid Price')
ax1.set_xlabel('Order Size')
ax1.set_ylabel('Price ($)')
ax1.set_title('Traditional Order Book\n(Discrete orders at specific prices)')
ax1.legend()
ax1.set_xlim(-50, 50)

# AMM visualization
ax2 = axes[1]
x = np.linspace(50, 200, 100)  # Token X reserves
k = 10000  # Constant product
y = k / x  # Token Y reserves
price = y / x  # Price of X in terms of Y

ax2.plot(x, price, 'b-', linewidth=2, label='AMM Price Curve')
ax2.axvline(x=100, color='green', linestyle='--', alpha=0.7, label='Current reserves')
ax2.scatter([100], [k/100/100], color='red', s=100, zorder=5, label='Current price')
ax2.set_xlabel('Token X Reserves')
ax2.set_ylabel('Price (Y per X)')
ax2.set_title('AMM Price Curve\n(Continuous pricing via formula)')
ax2.legend()

plt.tight_layout()
plt.show()

print("\nKey Differences:")
print("  Order Book: Discrete prices, requires active market makers")
print("  AMM: Continuous prices, liquidity always available")

## Section 3: The Constant Product Formula

### x * y = k

The most common AMM formula (used by Uniswap V2) is the **constant product formula**:

$$x \cdot y = k$$

Where:
- **x** = reserves of token X in the pool
- **y** = reserves of token Y in the pool
- **k** = constant (the "invariant")

**Key insight:** After any trade, the product of reserves must remain constant.

### Price Derivation

The **spot price** (price for infinitesimally small trades) is:

$$Price_{X} = \frac{y}{x}$$

This is the price of token X in terms of token Y.

In [ ]:
# Demonstrate the constant product formula

print("CONSTANT PRODUCT FORMULA: x * y = k")
print("=" * 60)

# Initial pool state
x = 100  # 100 ETH
y = 200000  # 200,000 USDC
k = x * y

print(f"\nInitial Pool State:")
print(f"  ETH reserves (x):  {x:,}")
print(f"  USDC reserves (y): {y:,}")
print(f"  Constant k:        {k:,}")
print(f"  Spot price:        {y/x:,.2f} USDC per ETH")

# Simulate a swap: Buy 10 ETH
eth_to_buy = 10
new_x = x - eth_to_buy  # Pool loses ETH
new_y = k / new_x  # Calculate new USDC reserves to maintain k
usdc_paid = new_y - y  # User pays this much USDC

print(f"\nAfter buying {eth_to_buy} ETH:")
print(f"  New ETH reserves:  {new_x:,}")
print(f"  New USDC reserves: {new_y:,.2f}")
print(f"  New constant k:    {new_x * new_y:,.2f} (unchanged!)")
print(f"  USDC paid:         {usdc_paid:,.2f}")
print(f"  Effective price:   {usdc_paid/eth_to_buy:,.2f} USDC per ETH")
print(f"  New spot price:    {new_y/new_x:,.2f} USDC per ETH")

print(f"\nNotice:")
print(f"  - Original spot price: {y/x:,.2f} USDC/ETH")
print(f"  - Effective price paid: {usdc_paid/eth_to_buy:,.2f} USDC/ETH")
print(f"  - Price increased by: {(usdc_paid/eth_to_buy)/(y/x)*100-100:.2f}%")
print(f"  - This difference is called SLIPPAGE or PRICE IMPACT")

In [ ]:
# Visualize the constant product curve

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: The bonding curve
ax1 = axes[0]
x_range = np.linspace(20, 200, 200)
y_range = k / x_range

ax1.plot(x_range, y_range, 'b-', linewidth=2, label='x * y = k')
ax1.scatter([x], [y], color='green', s=150, zorder=5, label=f'Before: ({x}, {y:,})')
ax1.scatter([new_x], [new_y], color='red', s=150, zorder=5, label=f'After: ({new_x}, {new_y:,.0f})')

# Draw the trade path
ax1.annotate('', xy=(new_x, new_y), xytext=(x, y),
             arrowprops=dict(arrowstyle='->', color='purple', lw=2))

ax1.set_xlabel('ETH Reserves (x)')
ax1.set_ylabel('USDC Reserves (y)')
ax1.set_title('Constant Product Curve\nTrade moves along the curve')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Price vs reserves
ax2 = axes[1]
prices = y_range / x_range

ax2.plot(x_range, prices, 'b-', linewidth=2)
ax2.scatter([x], [y/x], color='green', s=150, zorder=5, label=f'Before: {y/x:,.0f} USDC/ETH')
ax2.scatter([new_x], [new_y/new_x], color='red', s=150, zorder=5, label=f'After: {new_y/new_x:,.0f} USDC/ETH')
ax2.fill_between([new_x, x], [0, 0], [max(prices), max(prices)], alpha=0.2, color='purple', label='Trade region')

ax2.set_xlabel('ETH Reserves (x)')
ax2.set_ylabel('Price (USDC per ETH)')
ax2.set_title('Price vs ETH Reserves\nBuying ETH increases price')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Section 4: Implementing an AMM Class

Let's build a complete Uniswap-style AMM with:
- Adding liquidity
- Removing liquidity
- Swapping tokens
- LP (Liquidity Provider) tokens

In [ ]:
@dataclass
class SwapResult:
    """Result of a swap operation."""
    amount_in: float
    amount_out: float
    price_impact: float
    effective_price: float
    fee_paid: float


class ConstantProductAMM:
    """
    A Uniswap V2-style Constant Product Automated Market Maker.
    
    Implements the x * y = k formula with:
    - Liquidity provision and withdrawal
    - Token swaps with fees
    - LP token tracking
    """
    
    def __init__(self, token_x_name: str = "ETH", token_y_name: str = "USDC", fee: float = 0.003):
        """
        Initialize an empty AMM pool.
        
        Args:
            token_x_name: Name of token X
            token_y_name: Name of token Y  
            fee: Trading fee (0.003 = 0.3%, like Uniswap V2)
        """
        self.token_x_name = token_x_name
        self.token_y_name = token_y_name
        self.fee = fee
        
        # Pool reserves
        self.reserve_x = 0.0
        self.reserve_y = 0.0
        
        # LP token tracking
        self.total_lp_tokens = 0.0
        self.lp_balances: Dict[str, float] = {}
        
        # History for analysis
        self.trade_history: List[Dict] = []
        self.price_history: List[float] = []
    
    @property
    def k(self) -> float:
        """The constant product invariant."""
        return self.reserve_x * self.reserve_y
    
    @property
    def spot_price_x(self) -> float:
        """Spot price of token X in terms of token Y."""
        if self.reserve_x == 0:
            return 0
        return self.reserve_y / self.reserve_x
    
    @property
    def spot_price_y(self) -> float:
        """Spot price of token Y in terms of token X."""
        if self.reserve_y == 0:
            return 0
        return self.reserve_x / self.reserve_y
    
    def add_liquidity(self, provider: str, amount_x: float, amount_y: float) -> float:
        """
        Add liquidity to the pool.
        
        Args:
            provider: Address/name of liquidity provider
            amount_x: Amount of token X to add
            amount_y: Amount of token Y to add
            
        Returns:
            Number of LP tokens minted
        """
        if self.total_lp_tokens == 0:
            # First liquidity provision - mint sqrt(x * y) LP tokens
            lp_tokens = np.sqrt(amount_x * amount_y)
        else:
            # Subsequent provisions - mint proportional to existing liquidity
            # Must provide tokens in current ratio
            ratio_x = amount_x / self.reserve_x
            ratio_y = amount_y / self.reserve_y
            
            # Use minimum ratio to determine LP tokens
            ratio = min(ratio_x, ratio_y)
            lp_tokens = ratio * self.total_lp_tokens
            
            # Adjust amounts to match ratio
            amount_x = ratio * self.reserve_x
            amount_y = ratio * self.reserve_y
        
        # Update reserves
        self.reserve_x += amount_x
        self.reserve_y += amount_y
        
        # Mint LP tokens
        self.total_lp_tokens += lp_tokens
        self.lp_balances[provider] = self.lp_balances.get(provider, 0) + lp_tokens
        
        # Record price
        self.price_history.append(self.spot_price_x)
        
        return lp_tokens
    
    def remove_liquidity(self, provider: str, lp_tokens: float) -> Tuple[float, float]:
        """
        Remove liquidity from the pool.
        
        Args:
            provider: Address/name of liquidity provider
            lp_tokens: Number of LP tokens to burn
            
        Returns:
            Tuple of (amount_x, amount_y) returned to provider
        """
        if provider not in self.lp_balances or self.lp_balances[provider] < lp_tokens:
            raise ValueError(f"Insufficient LP tokens for {provider}")
        
        # Calculate share of pool
        share = lp_tokens / self.total_lp_tokens
        
        # Calculate amounts to return
        amount_x = share * self.reserve_x
        amount_y = share * self.reserve_y
        
        # Update reserves
        self.reserve_x -= amount_x
        self.reserve_y -= amount_y
        
        # Burn LP tokens
        self.total_lp_tokens -= lp_tokens
        self.lp_balances[provider] -= lp_tokens
        
        return amount_x, amount_y
    
    def get_amount_out(self, amount_in: float, is_x_to_y: bool) -> float:
        """
        Calculate output amount for a given input (with fees).
        
        Uses the formula: amount_out = (reserve_out * amount_in * (1-fee)) / (reserve_in + amount_in * (1-fee))
        """
        if is_x_to_y:
            reserve_in, reserve_out = self.reserve_x, self.reserve_y
        else:
            reserve_in, reserve_out = self.reserve_y, self.reserve_x
        
        amount_in_with_fee = amount_in * (1 - self.fee)
        amount_out = (reserve_out * amount_in_with_fee) / (reserve_in + amount_in_with_fee)
        
        return amount_out
    
    def swap_x_for_y(self, amount_x_in: float) -> SwapResult:
        """
        Swap token X for token Y.
        
        Args:
            amount_x_in: Amount of token X to swap
            
        Returns:
            SwapResult with details of the swap
        """
        # Calculate output
        amount_y_out = self.get_amount_out(amount_x_in, is_x_to_y=True)
        
        # Calculate price impact
        spot_price_before = self.spot_price_x
        effective_price = amount_y_out / amount_x_in
        price_impact = (spot_price_before - effective_price) / spot_price_before
        
        # Calculate fee
        fee_paid = amount_x_in * self.fee
        
        # Update reserves
        self.reserve_x += amount_x_in
        self.reserve_y -= amount_y_out
        
        # Record trade
        self.trade_history.append({
            'type': f'{self.token_x_name} -> {self.token_y_name}',
            'amount_in': amount_x_in,
            'amount_out': amount_y_out,
            'price_impact': price_impact,
            'spot_price_after': self.spot_price_x
        })
        self.price_history.append(self.spot_price_x)
        
        return SwapResult(
            amount_in=amount_x_in,
            amount_out=amount_y_out,
            price_impact=price_impact,
            effective_price=effective_price,
            fee_paid=fee_paid
        )
    
    def swap_y_for_x(self, amount_y_in: float) -> SwapResult:
        """
        Swap token Y for token X.
        
        Args:
            amount_y_in: Amount of token Y to swap
            
        Returns:
            SwapResult with details of the swap
        """
        # Calculate output
        amount_x_out = self.get_amount_out(amount_y_in, is_x_to_y=False)
        
        # Calculate price impact (for buying X, price goes up)
        spot_price_before = self.spot_price_x
        effective_price = amount_y_in / amount_x_out
        price_impact = (effective_price - spot_price_before) / spot_price_before
        
        # Calculate fee
        fee_paid = amount_y_in * self.fee
        
        # Update reserves
        self.reserve_y += amount_y_in
        self.reserve_x -= amount_x_out
        
        # Record trade
        self.trade_history.append({
            'type': f'{self.token_y_name} -> {self.token_x_name}',
            'amount_in': amount_y_in,
            'amount_out': amount_x_out,
            'price_impact': price_impact,
            'spot_price_after': self.spot_price_x
        })
        self.price_history.append(self.spot_price_x)
        
        return SwapResult(
            amount_in=amount_y_in,
            amount_out=amount_x_out,
            price_impact=price_impact,
            effective_price=effective_price,
            fee_paid=fee_paid
        )
    
    def __str__(self) -> str:
        return f"""
{'='*60}
AMM Pool: {self.token_x_name}/{self.token_y_name}
{'='*60}
Reserves:
  {self.token_x_name}: {self.reserve_x:,.4f}
  {self.token_y_name}: {self.reserve_y:,.4f}
  k (constant): {self.k:,.2f}

Prices:
  1 {self.token_x_name} = {self.spot_price_x:,.4f} {self.token_y_name}
  1 {self.token_y_name} = {self.spot_price_y:,.6f} {self.token_x_name}

Pool Stats:
  Fee: {self.fee*100:.2f}%
  Total LP Tokens: {self.total_lp_tokens:,.4f}
  Total Trades: {len(self.trade_history)}
{'='*60}
"""


print("ConstantProductAMM class defined successfully!")

In [ ]:
# Test our AMM implementation

print("TESTING AMM IMPLEMENTATION")
print("=" * 60)

# Create a new AMM pool
pool = ConstantProductAMM("ETH", "USDC", fee=0.003)

# Add initial liquidity
print("\n1. Adding initial liquidity...")
lp_tokens_alice = pool.add_liquidity("Alice", amount_x=100, amount_y=200000)
print(f"   Alice deposited: 100 ETH + 200,000 USDC")
print(f"   Alice received: {lp_tokens_alice:,.2f} LP tokens")

print(pool)

# Perform a swap
print("\n2. Bob swaps 10 ETH for USDC...")
result = pool.swap_x_for_y(10)
print(f"   Amount in:      10 ETH")
print(f"   Amount out:     {result.amount_out:,.2f} USDC")
print(f"   Effective price: {result.effective_price:,.2f} USDC/ETH")
print(f"   Price impact:   {result.price_impact*100:.2f}%")
print(f"   Fee paid:       {result.fee_paid:.4f} ETH")

print(pool)

# Another swap in opposite direction
print("\n3. Charlie swaps 5,000 USDC for ETH...")
result2 = pool.swap_y_for_x(5000)
print(f"   Amount in:      5,000 USDC")
print(f"   Amount out:     {result2.amount_out:,.4f} ETH")
print(f"   Effective price: {result2.effective_price:,.2f} USDC/ETH")
print(f"   Price impact:   {result2.price_impact*100:.2f}%")

print(pool)

## Section 5: Price Impact and Slippage

**Price Impact** (or slippage) is the difference between:
- The **spot price** before the trade
- The **effective price** you actually receive

Larger trades relative to pool size = more slippage.

This is why liquidity depth matters!

In [ ]:
# Analyze price impact for different trade sizes

def analyze_price_impact(pool_size_eth: float, pool_size_usdc: float, trade_sizes: List[float]):
    """
    Analyze price impact for various trade sizes.
    """
    results = []
    
    for trade_size in trade_sizes:
        # Create fresh pool for each analysis
        pool = ConstantProductAMM("ETH", "USDC", fee=0.003)
        pool.add_liquidity("LP", pool_size_eth, pool_size_usdc)
        
        spot_price = pool.spot_price_x
        result = pool.swap_x_for_y(trade_size)
        
        trade_pct = (trade_size / pool_size_eth) * 100
        
        results.append({
            'trade_size_eth': trade_size,
            'trade_pct_of_pool': trade_pct,
            'spot_price': spot_price,
            'effective_price': result.effective_price,
            'usdc_received': result.amount_out,
            'price_impact_pct': result.price_impact * 100
        })
    
    return pd.DataFrame(results)


# Analyze price impact
print("PRICE IMPACT ANALYSIS")
print("=" * 70)
print("\nPool: 1,000 ETH / 2,000,000 USDC (spot price: 2,000 USDC/ETH)")
print("\nSelling ETH for USDC at various sizes:\n")

trade_sizes = [1, 5, 10, 25, 50, 100, 200, 500]
df_impact = analyze_price_impact(1000, 2000000, trade_sizes)

# Format and display
df_display = df_impact.copy()
df_display['trade_size_eth'] = df_display['trade_size_eth'].apply(lambda x: f"{x:,.0f}")
df_display['trade_pct_of_pool'] = df_display['trade_pct_of_pool'].apply(lambda x: f"{x:.1f}%")
df_display['spot_price'] = df_display['spot_price'].apply(lambda x: f"${x:,.0f}")
df_display['effective_price'] = df_display['effective_price'].apply(lambda x: f"${x:,.2f}")
df_display['usdc_received'] = df_display['usdc_received'].apply(lambda x: f"${x:,.0f}")
df_display['price_impact_pct'] = df_display['price_impact_pct'].apply(lambda x: f"{x:.2f}%")

df_display.columns = ['Trade Size (ETH)', '% of Pool', 'Spot Price', 'Effective Price', 'USDC Received', 'Price Impact']
print(df_display.to_string(index=False))

print("\nKey Insight: Price impact grows non-linearly with trade size!")

In [ ]:
# Visualize price impact

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Price impact vs trade size
ax1 = axes[0]
trade_sizes_fine = np.linspace(1, 500, 100)
impacts = []

for size in trade_sizes_fine:
    pool = ConstantProductAMM("ETH", "USDC", fee=0.003)
    pool.add_liquidity("LP", 1000, 2000000)
    result = pool.swap_x_for_y(size)
    impacts.append(result.price_impact * 100)

ax1.plot(trade_sizes_fine, impacts, 'b-', linewidth=2)
ax1.axhline(y=1, color='orange', linestyle='--', label='1% slippage threshold')
ax1.axhline(y=5, color='red', linestyle='--', label='5% slippage threshold')
ax1.fill_between(trade_sizes_fine, 0, 1, alpha=0.2, color='green', label='Low slippage zone')
ax1.set_xlabel('Trade Size (ETH)')
ax1.set_ylabel('Price Impact (%)')
ax1.set_title('Price Impact vs Trade Size\n(Pool: 1,000 ETH / 2M USDC)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Effective price vs trade size
ax2 = axes[1]
effective_prices = []

for size in trade_sizes_fine:
    pool = ConstantProductAMM("ETH", "USDC", fee=0.003)
    pool.add_liquidity("LP", 1000, 2000000)
    result = pool.swap_x_for_y(size)
    effective_prices.append(result.effective_price)

ax2.plot(trade_sizes_fine, effective_prices, 'b-', linewidth=2, label='Effective price')
ax2.axhline(y=2000, color='green', linestyle='--', label='Spot price ($2,000)')
ax2.set_xlabel('Trade Size (ETH)')
ax2.set_ylabel('Effective Price (USDC/ETH)')
ax2.set_title('Effective Price vs Trade Size\n(Selling ETH pushes price down)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Compare price impact across different pool sizes (liquidity depth)

print("\nLIQUIDITY DEPTH COMPARISON")
print("=" * 70)
print("\nHow does pool size affect price impact for a 50 ETH trade?\n")

pool_sizes = [100, 500, 1000, 5000, 10000]  # ETH in pool
trade_size = 50  # ETH

results = []
for pool_eth in pool_sizes:
    pool_usdc = pool_eth * 2000  # Same price ratio
    pool = ConstantProductAMM("ETH", "USDC", fee=0.003)
    pool.add_liquidity("LP", pool_eth, pool_usdc)
    result = pool.swap_x_for_y(trade_size)
    
    results.append({
        'pool_eth': pool_eth,
        'pool_usdc': pool_usdc,
        'tvl': pool_eth * 2000 * 2,  # Total Value Locked
        'price_impact': result.price_impact * 100,
        'usdc_received': result.amount_out
    })

df_depth = pd.DataFrame(results)

fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.bar(range(len(pool_sizes)), df_depth['price_impact'], color='steelblue', alpha=0.8)
ax.set_xticks(range(len(pool_sizes)))
ax.set_xticklabels([f"{p:,} ETH\n(${p*4000:,} TVL)" for p in pool_sizes])
ax.set_ylabel('Price Impact (%)')
ax.set_title(f'Price Impact of {trade_size} ETH Swap vs Pool Size\n(Deeper liquidity = less slippage)')
ax.axhline(y=1, color='orange', linestyle='--', label='1% threshold')

# Add value labels on bars
for bar, impact in zip(bars, df_depth['price_impact']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
            f'{impact:.1f}%', ha='center', fontsize=10)

ax.legend()
plt.tight_layout()
plt.show()

print("Key Insight: 10x more liquidity roughly means 10x less price impact!")

## Section 6: Impermanent Loss

**Impermanent Loss (IL)** is the opportunity cost of providing liquidity vs simply holding the tokens.

When you provide liquidity:
- If prices change, arbitrageurs rebalance your position
- You end up with more of the depreciating token
- You end up with less of the appreciating token
- This is worse than just holding!

### The Formula

For a price change ratio r (new_price / initial_price):

$$IL = \frac{2\sqrt{r}}{1 + r} - 1$$

The loss is "impermanent" because if prices return to the original ratio, the loss disappears.

In [ ]:
def calculate_impermanent_loss(price_ratio: float) -> float:
    """
    Calculate impermanent loss for a given price change ratio.
    
    Args:
        price_ratio: new_price / initial_price
        
    Returns:
        Impermanent loss as a decimal (negative = loss)
    """
    return 2 * np.sqrt(price_ratio) / (1 + price_ratio) - 1


def simulate_lp_position(initial_eth: float, initial_usdc: float, 
                         price_change_pct: float) -> Dict:
    """
    Simulate LP position value vs holding after price change.
    """
    initial_price = initial_usdc / initial_eth
    new_price = initial_price * (1 + price_change_pct / 100)
    
    # Value if just holding
    hold_value = initial_eth * new_price + initial_usdc
    
    # Value as LP (after arbitrage rebalancing)
    # After arbitrage: new_eth * new_usdc = k (constant)
    # And: new_usdc / new_eth = new_price
    k = initial_eth * initial_usdc
    new_eth = np.sqrt(k / new_price)
    new_usdc = np.sqrt(k * new_price)
    lp_value = new_eth * new_price + new_usdc
    
    # Impermanent loss
    il = (lp_value - hold_value) / hold_value
    
    return {
        'initial_eth': initial_eth,
        'initial_usdc': initial_usdc,
        'initial_price': initial_price,
        'new_price': new_price,
        'price_change_pct': price_change_pct,
        'new_eth': new_eth,
        'new_usdc': new_usdc,
        'hold_value': hold_value,
        'lp_value': lp_value,
        'impermanent_loss': il,
        'il_usd': lp_value - hold_value
    }


# Demonstrate impermanent loss
print("IMPERMANENT LOSS DEMONSTRATION")
print("=" * 70)

initial_eth = 10
initial_usdc = 20000  # $2,000/ETH

print(f"\nInitial Position:")
print(f"  {initial_eth} ETH + {initial_usdc:,} USDC")
print(f"  Initial price: ${initial_usdc/initial_eth:,}/ETH")
print(f"  Total value: ${initial_eth * 2000 + initial_usdc:,}\n")

# Test various price changes
price_changes = [-50, -25, 0, 25, 50, 100, 200]

print(f"{'Price Change':<15} {'New Price':<12} {'HOLD Value':<15} {'LP Value':<15} {'IL':<10} {'IL ($)':<12}")
print("-" * 80)

for pct in price_changes:
    result = simulate_lp_position(initial_eth, initial_usdc, pct)
    print(f"{pct:+.0f}%{'':<10} ${result['new_price']:,.0f}{'':<5} ${result['hold_value']:,.0f}{'':<6} ${result['lp_value']:,.0f}{'':<6} {result['impermanent_loss']*100:+.2f}%{'':<3} ${result['il_usd']:+,.0f}")

print("\nKey Observations:")
print("  1. IL is always negative (you always lose vs holding)")
print("  2. IL is the same whether price goes up OR down by same %")
print("  3. IL accelerates with larger price moves")
print("  4. Fees earned may compensate for IL (that's why LPs do it!)")

In [ ]:
# Visualize impermanent loss

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: IL vs price ratio
ax1 = axes[0]
price_ratios = np.linspace(0.1, 5, 200)
il_values = [calculate_impermanent_loss(r) * 100 for r in price_ratios]

ax1.plot(price_ratios, il_values, 'b-', linewidth=2)
ax1.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax1.axvline(x=1, color='green', linestyle='--', alpha=0.7, label='No price change')
ax1.fill_between(price_ratios, il_values, 0, alpha=0.3, color='red')

# Mark key points
key_ratios = [0.5, 0.75, 1.25, 1.5, 2, 3, 4]
for r in key_ratios:
    il = calculate_impermanent_loss(r) * 100
    ax1.scatter([r], [il], color='red', s=50, zorder=5)
    ax1.annotate(f'{il:.1f}%', (r, il), textcoords='offset points', 
                 xytext=(0, 10), ha='center', fontsize=9)

ax1.set_xlabel('Price Ratio (new_price / initial_price)')
ax1.set_ylabel('Impermanent Loss (%)')
ax1.set_title('Impermanent Loss vs Price Change\n(Always negative!)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-60, 5)

# Plot 2: Value comparison
ax2 = axes[1]
price_changes = np.linspace(-80, 300, 200)
hold_values = []
lp_values = []

for pct in price_changes:
    result = simulate_lp_position(10, 20000, pct)
    hold_values.append(result['hold_value'])
    lp_values.append(result['lp_value'])

ax2.plot(price_changes, hold_values, 'g-', linewidth=2, label='HODL Value')
ax2.plot(price_changes, lp_values, 'b-', linewidth=2, label='LP Value')
ax2.fill_between(price_changes, lp_values, hold_values, alpha=0.3, color='red', label='Impermanent Loss')
ax2.axvline(x=0, color='black', linestyle='--', alpha=0.5)

ax2.set_xlabel('Price Change (%)')
ax2.set_ylabel('Portfolio Value ($)')
ax2.set_title('LP Value vs HODL Value\n(Initial: 10 ETH + 20,000 USDC)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nImpermanent Loss Reference Table:")
print(f"  Price change of 25% (either direction): {calculate_impermanent_loss(1.25)*100:.2f}% IL")
print(f"  Price change of 50% (either direction): {calculate_impermanent_loss(1.5)*100:.2f}% IL")
print(f"  Price change of 100% (2x): {calculate_impermanent_loss(2)*100:.2f}% IL")
print(f"  Price change of 200% (3x): {calculate_impermanent_loss(3)*100:.2f}% IL")
print(f"  Price change of 400% (5x): {calculate_impermanent_loss(5)*100:.2f}% IL")

## Section 7: Arbitrage Opportunities

**Arbitrage** keeps AMM prices aligned with external markets.

When the AMM price differs from the "true" market price:
1. Arbitrageurs buy cheap and sell expensive
2. This moves the AMM price toward the market price
3. Arbitrageurs profit, but they also provide a service!

Let's simulate arbitrage:

In [ ]:
def calculate_arbitrage(pool: ConstantProductAMM, external_price: float) -> Dict:
    """
    Calculate optimal arbitrage trade given external market price.
    
    Args:
        pool: The AMM pool
        external_price: External market price (Y per X)
        
    Returns:
        Dictionary with arbitrage details
    """
    amm_price = pool.spot_price_x
    price_diff_pct = (amm_price - external_price) / external_price * 100
    
    if abs(price_diff_pct) < 0.1:  # Less than 0.1% difference
        return {
            'opportunity': False,
            'direction': None,
            'optimal_trade': 0,
            'profit': 0,
            'price_diff_pct': price_diff_pct
        }
    
    # Determine arbitrage direction
    if amm_price > external_price:
        # AMM price is high - sell X to AMM, buy X cheaper externally
        direction = "sell_x"
        # Calculate optimal trade size using calculus
        # After selling dx of X: new_price = (y + dy) / (x - dx) = external_price
        # With constant product: (x - dx)(y + dy) = xy
        k = pool.k
        x, y = pool.reserve_x, pool.reserve_y
        # Optimal: new_x = sqrt(k / external_price)
        new_x = np.sqrt(k / external_price)
        optimal_dx = x - new_x
        
        if optimal_dx > 0:
            # Simulate the trade
            test_pool = ConstantProductAMM(fee=pool.fee)
            test_pool.reserve_x = pool.reserve_x
            test_pool.reserve_y = pool.reserve_y
            test_pool.total_lp_tokens = pool.total_lp_tokens
            
            result = test_pool.swap_x_for_y(optimal_dx)
            # Profit = USDC received - cost to buy ETH externally
            profit = result.amount_out - optimal_dx * external_price
        else:
            optimal_dx = 0
            profit = 0
            
    else:
        # AMM price is low - buy X from AMM, sell X externally for more
        direction = "buy_x"
        k = pool.k
        x, y = pool.reserve_x, pool.reserve_y
        new_x = np.sqrt(k / external_price)
        optimal_dx = new_x - x  # We're buying, so new_x > x
        
        if optimal_dx > 0:
            # Calculate USDC needed to buy this much ETH
            test_pool = ConstantProductAMM(fee=pool.fee)
            test_pool.reserve_x = pool.reserve_x
            test_pool.reserve_y = pool.reserve_y
            test_pool.total_lp_tokens = pool.total_lp_tokens
            
            # How much USDC to get optimal_dx ETH?
            # Use binary search to find it
            usdc_needed = 0
            for _ in range(50):  # Binary search iterations
                mid = (0 + pool.reserve_y) / 2
                test = test_pool.get_amount_out(mid, is_x_to_y=False)
                if test < optimal_dx:
                    usdc_needed = mid
                    break
            
            # Rough calculation
            new_y = k / new_x
            usdc_needed = new_y - y
            
            # Profit = ETH sold externally - USDC spent
            eth_received = x - new_x  # This is negative (we're taking ETH out)
            profit = abs(eth_received) * external_price - usdc_needed
            optimal_dx = abs(eth_received)
        else:
            optimal_dx = 0
            profit = 0
    
    return {
        'opportunity': profit > 0,
        'direction': direction,
        'optimal_trade': optimal_dx,
        'profit': max(0, profit),
        'price_diff_pct': price_diff_pct,
        'amm_price': amm_price,
        'external_price': external_price
    }


# Demonstrate arbitrage
print("ARBITRAGE SIMULATION")
print("=" * 70)

# Create pool with 100 ETH at $2000 each
arb_pool = ConstantProductAMM("ETH", "USDC", fee=0.003)
arb_pool.add_liquidity("LP", 100, 200000)

print(f"\nInitial AMM State:")
print(f"  Reserves: {arb_pool.reserve_x} ETH / {arb_pool.reserve_y:,} USDC")
print(f"  AMM Price: ${arb_pool.spot_price_x:,.2f}/ETH")

# Scenario 1: External price is higher
print(f"\n" + "-"*70)
print("Scenario 1: External market price rises to $2,200/ETH")
print("-"*70)

external_price = 2200
arb = calculate_arbitrage(arb_pool, external_price)

print(f"  AMM Price: ${arb['amm_price']:,.2f}")
print(f"  External Price: ${arb['external_price']:,.2f}")
print(f"  Price Difference: {arb['price_diff_pct']:+.2f}%")
print(f"  Arbitrage Direction: {arb['direction']}")
print(f"  Optimal Trade: {arb['optimal_trade']:.4f} ETH")
print(f"  Estimated Profit: ${arb['profit']:,.2f}")

if arb['direction'] == 'buy_x':
    print(f"\n  Strategy: Buy cheap ETH from AMM, sell at ${external_price} externally")

# Execute the arbitrage
if arb['opportunity']:
    print(f"\n  Executing arbitrage...")
    result = arb_pool.swap_y_for_x(arb['optimal_trade'] * arb_pool.spot_price_x * 1.1)  # Rough USDC amount
    print(f"  New AMM Price: ${arb_pool.spot_price_x:,.2f}/ETH")
    print(f"  Price now closer to external market!")

In [ ]:
# Visualize arbitrage dynamics

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Simulate price convergence through arbitrage
ax1 = axes[0]

# Create fresh pool
sim_pool = ConstantProductAMM("ETH", "USDC", fee=0.003)
sim_pool.add_liquidity("LP", 1000, 2000000)

external_prices = [2000]  # Start at AMM price
amm_prices = [sim_pool.spot_price_x]

# Simulate external price jump and arbitrage response
np.random.seed(42)
for i in range(50):
    # External price random walk
    ext_change = np.random.normal(0, 20)
    external_prices.append(external_prices[-1] + ext_change)
    
    # Arbitrage moves AMM price toward external
    price_gap = external_prices[-1] - sim_pool.spot_price_x
    if abs(price_gap) > 5:  # Only arb if gap > $5
        # Rough simulation of arbitrage effect
        if price_gap > 0:  # External higher, buy from AMM
            trade_size = min(abs(price_gap) / 100, 50)  # Conservative trade
            try:
                sim_pool.swap_y_for_x(trade_size * sim_pool.spot_price_x)
            except:
                pass
        else:  # External lower, sell to AMM
            trade_size = min(abs(price_gap) / 100, 50)
            try:
                sim_pool.swap_x_for_y(trade_size)
            except:
                pass
    
    amm_prices.append(sim_pool.spot_price_x)

ax1.plot(external_prices, 'g-', linewidth=2, label='External Market Price', alpha=0.8)
ax1.plot(amm_prices, 'b-', linewidth=2, label='AMM Price')
ax1.set_xlabel('Time Step')
ax1.set_ylabel('Price (USDC/ETH)')
ax1.set_title('Arbitrage Keeps AMM Price Aligned\n(AMM follows external market)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Arbitrage profit vs price difference
ax2 = axes[1]

price_diffs = np.linspace(-10, 10, 50)  # % difference from AMM
profits = []

base_price = 2000
for diff in price_diffs:
    test_pool = ConstantProductAMM("ETH", "USDC", fee=0.003)
    test_pool.add_liquidity("LP", 1000, 2000000)
    ext_price = base_price * (1 + diff/100)
    arb = calculate_arbitrage(test_pool, ext_price)
    profits.append(arb['profit'])

ax2.plot(price_diffs, profits, 'b-', linewidth=2)
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax2.axvline(x=0, color='green', linestyle='--', alpha=0.7, label='No price difference')
ax2.fill_between(price_diffs, profits, 0, where=[p > 0 for p in profits], 
                  alpha=0.3, color='green', label='Profitable arbitrage')

ax2.set_xlabel('Price Difference from AMM (%)')
ax2.set_ylabel('Arbitrage Profit ($)')
ax2.set_title('Arbitrage Profit vs Price Discrepancy\n(Pool: 1000 ETH / 2M USDC)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("  1. Arbitrageurs constantly monitor AMM vs external prices")
print("  2. Price differences create risk-free profit opportunities")
print("  3. Arbitrage quickly eliminates price discrepancies")
print("  4. Fees eat into arbitrage profits (need sufficient price gap)")
print("  5. This is how AMMs discover and maintain accurate prices!")

## Section 8: Multi-hop Swaps (Routing)

What if you want to trade tokens that don't have a direct pair?

Example: Trade LINK for UNI
- No direct LINK/UNI pool
- But there's LINK/ETH and ETH/UNI pools
- Route: LINK -> ETH -> UNI

This is called **multi-hop routing**.

In [ ]:
class MultiPoolRouter:
    """
    Router for multi-hop swaps across multiple AMM pools.
    """
    
    def __init__(self):
        self.pools: Dict[str, ConstantProductAMM] = {}
    
    def add_pool(self, pool: ConstantProductAMM) -> None:
        """Add a pool to the router."""
        pair_name = f"{pool.token_x_name}/{pool.token_y_name}"
        self.pools[pair_name] = pool
    
    def get_pool(self, token_a: str, token_b: str) -> Optional[Tuple[ConstantProductAMM, bool]]:
        """
        Find pool for token pair.
        
        Returns:
            Tuple of (pool, is_x_to_y) or None if not found
        """
        key1 = f"{token_a}/{token_b}"
        key2 = f"{token_b}/{token_a}"
        
        if key1 in self.pools:
            return self.pools[key1], True
        elif key2 in self.pools:
            return self.pools[key2], False
        return None
    
    def find_route(self, token_in: str, token_out: str, 
                   intermediate_tokens: List[str] = None) -> List[str]:
        """
        Find a route from token_in to token_out.
        
        For simplicity, tries direct route first, then common intermediates.
        """
        # Try direct route
        if self.get_pool(token_in, token_out):
            return [token_in, token_out]
        
        # Try intermediate tokens (default: ETH, USDC, USDT)
        intermediates = intermediate_tokens or ['ETH', 'USDC', 'USDT']
        
        for mid in intermediates:
            if mid == token_in or mid == token_out:
                continue
            if self.get_pool(token_in, mid) and self.get_pool(mid, token_out):
                return [token_in, mid, token_out]
        
        return []  # No route found
    
    def get_quote(self, amount_in: float, route: List[str]) -> Dict:
        """
        Get quote for a multi-hop swap without executing.
        """
        if len(route) < 2:
            return {'success': False, 'error': 'Invalid route'}
        
        current_amount = amount_in
        total_price_impact = 0
        hops = []
        
        for i in range(len(route) - 1):
            token_in = route[i]
            token_out = route[i + 1]
            
            pool_result = self.get_pool(token_in, token_out)
            if not pool_result:
                return {'success': False, 'error': f'No pool for {token_in}/{token_out}'}
            
            pool, is_x_to_y = pool_result
            
            # Calculate output without executing
            if is_x_to_y:
                amount_out = pool.get_amount_out(current_amount, is_x_to_y=True)
                spot_price = pool.spot_price_x
            else:
                amount_out = pool.get_amount_out(current_amount, is_x_to_y=False)
                spot_price = pool.spot_price_y
            
            effective_price = amount_out / current_amount if current_amount > 0 else 0
            price_impact = abs((spot_price - effective_price) / spot_price) if spot_price > 0 else 0
            total_price_impact += price_impact
            
            hops.append({
                'from': token_in,
                'to': token_out,
                'amount_in': current_amount,
                'amount_out': amount_out,
                'price_impact': price_impact
            })
            
            current_amount = amount_out
        
        return {
            'success': True,
            'route': route,
            'amount_in': amount_in,
            'amount_out': current_amount,
            'total_price_impact': total_price_impact,
            'hops': hops
        }
    
    def execute_swap(self, amount_in: float, route: List[str], 
                     min_amount_out: float = 0) -> Dict:
        """
        Execute a multi-hop swap.
        """
        quote = self.get_quote(amount_in, route)
        if not quote['success']:
            return quote
        
        if quote['amount_out'] < min_amount_out:
            return {
                'success': False,
                'error': f"Slippage too high. Expected {min_amount_out}, got {quote['amount_out']}"
            }
        
        # Execute each hop
        current_amount = amount_in
        for i in range(len(route) - 1):
            token_in = route[i]
            token_out = route[i + 1]
            pool, is_x_to_y = self.get_pool(token_in, token_out)
            
            if is_x_to_y:
                result = pool.swap_x_for_y(current_amount)
            else:
                result = pool.swap_y_for_x(current_amount)
            
            current_amount = result.amount_out
        
        return {
            'success': True,
            'amount_in': amount_in,
            'amount_out': current_amount,
            'route': ' -> '.join(route)
        }


print("MultiPoolRouter class defined successfully!")

In [ ]:
# Create a multi-pool ecosystem

print("MULTI-HOP SWAP DEMONSTRATION")
print("=" * 70)

# Create router
router = MultiPoolRouter()

# Create pools
eth_usdc = ConstantProductAMM("ETH", "USDC", fee=0.003)
eth_usdc.add_liquidity("LP1", 1000, 2000000)  # 1000 ETH at $2000

link_eth = ConstantProductAMM("LINK", "ETH", fee=0.003)
link_eth.add_liquidity("LP2", 50000, 1000)  # 50000 LINK, 1000 ETH (LINK = $40)

uni_eth = ConstantProductAMM("UNI", "ETH", fee=0.003)
uni_eth.add_liquidity("LP3", 100000, 1000)  # 100000 UNI, 1000 ETH (UNI = $20)

# Add pools to router
router.add_pool(eth_usdc)
router.add_pool(link_eth)
router.add_pool(uni_eth)

print("\nPools Created:")
print(f"  1. ETH/USDC: {eth_usdc.reserve_x} ETH / {eth_usdc.reserve_y:,} USDC")
print(f"     Price: ${eth_usdc.spot_price_x:,.2f}/ETH")
print(f"  2. LINK/ETH: {link_eth.reserve_x:,} LINK / {link_eth.reserve_y} ETH")
print(f"     Price: {link_eth.spot_price_x:.4f} ETH/LINK = ${link_eth.spot_price_x * 2000:.2f}/LINK")
print(f"  3. UNI/ETH: {uni_eth.reserve_x:,} UNI / {uni_eth.reserve_y} ETH")
print(f"     Price: {uni_eth.spot_price_x:.4f} ETH/UNI = ${uni_eth.spot_price_x * 2000:.2f}/UNI")

# Example 1: Direct swap (LINK -> ETH)
print(f"\n" + "-"*70)
print("Example 1: Direct Swap - 1000 LINK -> ETH")
print("-"*70)

route1 = router.find_route("LINK", "ETH")
print(f"  Route: {' -> '.join(route1)}")

quote1 = router.get_quote(1000, route1)
print(f"  Amount in: 1000 LINK")
print(f"  Amount out: {quote1['amount_out']:.4f} ETH")
print(f"  Price impact: {quote1['total_price_impact']*100:.2f}%")

# Example 2: Multi-hop swap (LINK -> UNI through ETH)
print(f"\n" + "-"*70)
print("Example 2: Multi-hop Swap - 1000 LINK -> UNI")
print("-"*70)

route2 = router.find_route("LINK", "UNI")
print(f"  Route: {' -> '.join(route2)}")

quote2 = router.get_quote(1000, route2)
print(f"  Amount in: 1000 LINK")
print(f"\n  Hop 1: LINK -> ETH")
print(f"    {quote2['hops'][0]['amount_in']:.2f} LINK -> {quote2['hops'][0]['amount_out']:.4f} ETH")
print(f"    Price impact: {quote2['hops'][0]['price_impact']*100:.2f}%")
print(f"\n  Hop 2: ETH -> UNI")
print(f"    {quote2['hops'][1]['amount_in']:.4f} ETH -> {quote2['hops'][1]['amount_out']:.2f} UNI")
print(f"    Price impact: {quote2['hops'][1]['price_impact']*100:.2f}%")
print(f"\n  Final: 1000 LINK -> {quote2['amount_out']:.2f} UNI")
print(f"  Total price impact: {quote2['total_price_impact']*100:.2f}%")

# Execute the swap
print(f"\n  Executing swap...")
result = router.execute_swap(1000, route2)
print(f"  Result: {result['amount_in']} LINK -> {result['amount_out']:.2f} UNI")

## Section 9: Challenge Exercises

Test your understanding with these challenges!

### Challenge 1: Fee Analysis

Calculate how much fee revenue LPs earn from trading volume. If a pool has $1M daily volume and 0.3% fee, how much do LPs earn per day? Per year? How does this compare to impermanent loss?

In [ ]:
# YOUR TURN: Complete this challenge!

print("CHALLENGE 1: Fee Revenue Analysis")
print("=" * 60)

# Parameters
daily_volume = 1_000_000  # $1M daily trading volume
fee_rate = 0.003  # 0.3% fee
pool_tvl = 10_000_000  # $10M total value locked

# TODO: Calculate daily and annual fee revenue
daily_fees = daily_volume * fee_rate
annual_fees = daily_fees * 365

print(f"\nPool Parameters:")
print(f"  Daily Volume: ${daily_volume:,}")
print(f"  Fee Rate: {fee_rate*100}%")
print(f"  Pool TVL: ${pool_tvl:,}")

print(f"\nFee Revenue:")
print(f"  Daily: ${daily_fees:,.0f}")
print(f"  Annual: ${annual_fees:,.0f}")

# TODO: Calculate APY for liquidity providers
fee_apy = (annual_fees / pool_tvl) * 100
print(f"  Fee APY: {fee_apy:.2f}%")

# TODO: Compare to impermanent loss
# If price changes by 50%, what's the IL?
price_change = 0.5  # 50% change
il = calculate_impermanent_loss(1 + price_change) * 100

print(f"\nImpermanent Loss Comparison:")
print(f"  If price changes by {price_change*100}%: {il:.2f}% IL")
print(f"  Fee APY would need to exceed {abs(il):.2f}% to compensate")

# TODO: How many days of fees to recover from 50% price move IL?
il_dollar = abs(il / 100) * pool_tvl
days_to_recover = il_dollar / daily_fees
print(f"  IL in dollars: ${il_dollar:,.0f}")
print(f"  Days of fees to recover: {days_to_recover:.0f} days")

### Challenge 2: Optimal Trade Splitting

Large trades have high price impact. Can you reduce price impact by splitting a large trade into smaller pieces? Simulate this and visualize the results.

In [ ]:
# YOUR TURN: Complete this challenge!

print("CHALLENGE 2: Trade Splitting Analysis")
print("=" * 60)

def execute_split_trade(pool_eth: float, pool_usdc: float, 
                        total_trade: float, num_splits: int) -> Dict:
    """
    Execute a trade split into multiple smaller trades.
    
    NOTE: In practice, this doesn't help on a single pool because
    the price moves with each trade. But it's educational!
    """
    pool = ConstantProductAMM("ETH", "USDC", fee=0.003)
    pool.add_liquidity("LP", pool_eth, pool_usdc)
    
    initial_price = pool.spot_price_x
    trade_size = total_trade / num_splits
    total_received = 0
    
    for i in range(num_splits):
        result = pool.swap_x_for_y(trade_size)
        total_received += result.amount_out
    
    effective_price = total_received / total_trade
    price_impact = (initial_price - effective_price) / initial_price
    
    return {
        'num_splits': num_splits,
        'total_trade': total_trade,
        'total_received': total_received,
        'effective_price': effective_price,
        'price_impact': price_impact * 100,
        'final_pool_price': pool.spot_price_x
    }

# Test different split strategies
total_eth = 100  # Total ETH to sell
splits_to_test = [1, 2, 5, 10, 20, 50, 100]

print(f"\nSelling {total_eth} ETH (Pool: 1000 ETH / 2M USDC)\n")
print(f"{'Splits':<10} {'USDC Received':<18} {'Effective Price':<18} {'Price Impact':<15}")
print("-" * 65)

results = []
for splits in splits_to_test:
    result = execute_split_trade(1000, 2000000, total_eth, splits)
    results.append(result)
    print(f"{splits:<10} ${result['total_received']:>14,.2f}   ${result['effective_price']:>14,.2f}   {result['price_impact']:>10.3f}%")

print("\nObservation: Splitting doesn't help on a single pool!")
print("The total price movement is determined by the final state, not the path.")
print("\nWhere splitting DOES help:")
print("  1. Trading across multiple pools (route splitting)")
print("  2. Time-weighted execution (TWAP) to avoid sandwich attacks")
print("  3. Trading during high-liquidity periods")

### Challenge 3: LP Profitability Simulation

Simulate being an LP for 30 days with random price movements and trading volume. Track:
- Fee income
- Impermanent loss
- Net P&L vs just holding

In [ ]:
# YOUR TURN: Complete this challenge!

print("CHALLENGE 3: LP Profitability Simulation")
print("=" * 60)

np.random.seed(42)

# Simulation parameters
days = 30
initial_eth = 10
initial_usdc = 20000
daily_volume_pct = 0.1  # 10% of pool traded daily
price_volatility = 0.03  # 3% daily volatility

# Track metrics
prices = [2000]  # Starting price
lp_values = []
hold_values = []
cumulative_fees = [0]

# Create pool
sim_pool = ConstantProductAMM("ETH", "USDC", fee=0.003)
lp_tokens = sim_pool.add_liquidity("LP", initial_eth, initial_usdc)

for day in range(days):
    # Random price change
    price_change = np.random.normal(0, price_volatility)
    new_price = prices[-1] * (1 + price_change)
    prices.append(new_price)
    
    # Simulate trading volume (random direction)
    trade_volume = sim_pool.reserve_x * daily_volume_pct
    
    # Random trades throughout the day
    num_trades = np.random.randint(5, 20)
    daily_fees = 0
    
    for _ in range(num_trades):
        trade_size = trade_volume / num_trades * np.random.uniform(0.5, 1.5)
        if np.random.random() > 0.5:
            # Sell ETH
            if trade_size < sim_pool.reserve_x * 0.1:  # Limit trade size
                result = sim_pool.swap_x_for_y(trade_size)
                daily_fees += result.fee_paid * sim_pool.spot_price_x
        else:
            # Buy ETH
            usdc_amount = trade_size * sim_pool.spot_price_x
            if usdc_amount < sim_pool.reserve_y * 0.1:
                result = sim_pool.swap_y_for_x(usdc_amount)
                daily_fees += result.fee_paid
    
    cumulative_fees.append(cumulative_fees[-1] + daily_fees)
    
    # Calculate values
    lp_value = sim_pool.reserve_x * new_price + sim_pool.reserve_y
    hold_value = initial_eth * new_price + initial_usdc
    lp_values.append(lp_value)
    hold_values.append(hold_value)

# Calculate final metrics
final_lp_value = lp_values[-1]
final_hold_value = hold_values[-1]
total_fees = cumulative_fees[-1]
impermanent_loss = final_lp_value - final_hold_value
net_pnl = final_lp_value + total_fees - (initial_eth * 2000 + initial_usdc)

print(f"\nSimulation Results ({days} days):")
print(f"  Initial Value: ${initial_eth * 2000 + initial_usdc:,.0f}")
print(f"  Final LP Value: ${final_lp_value:,.0f}")
print(f"  HODL Value: ${final_hold_value:,.0f}")
print(f"  Fees Earned: ${total_fees:,.0f}")
print(f"  Impermanent Loss: ${impermanent_loss:,.0f}")
print(f"  Net P&L: ${net_pnl:+,.0f}")
print(f"  Price Change: {(prices[-1]/prices[0]-1)*100:+.1f}%")

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Price over time
axes[0, 0].plot(prices, 'b-', linewidth=2)
axes[0, 0].set_title('ETH Price Over Time')
axes[0, 0].set_xlabel('Day')
axes[0, 0].set_ylabel('Price (USDC)')
axes[0, 0].grid(True, alpha=0.3)

# LP vs HODL value
axes[0, 1].plot(lp_values, 'b-', linewidth=2, label='LP Value')
axes[0, 1].plot(hold_values, 'g--', linewidth=2, label='HODL Value')
axes[0, 1].set_title('LP Value vs HODL Value')
axes[0, 1].set_xlabel('Day')
axes[0, 1].set_ylabel('Value (USDC)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Cumulative fees
axes[1, 0].plot(cumulative_fees, 'g-', linewidth=2)
axes[1, 0].set_title('Cumulative Fee Revenue')
axes[1, 0].set_xlabel('Day')
axes[1, 0].set_ylabel('Fees (USDC)')
axes[1, 0].grid(True, alpha=0.3)

# IL vs Fees
il_over_time = [lp - hold for lp, hold in zip(lp_values, hold_values)]
axes[1, 1].plot(il_over_time, 'r-', linewidth=2, label='Impermanent Loss')
axes[1, 1].plot(cumulative_fees[1:], 'g-', linewidth=2, label='Cumulative Fees')
net_over_time = [il + fee for il, fee in zip(il_over_time, cumulative_fees[1:])]
axes[1, 1].plot(net_over_time, 'b--', linewidth=2, label='Net (IL + Fees)')
axes[1, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
axes[1, 1].set_title('Impermanent Loss vs Fee Revenue')
axes[1, 1].set_xlabel('Day')
axes[1, 1].set_ylabel('USDC')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

In this notebook, you learned about **Automated Market Makers (AMMs)**:

### Key Concepts

1. **Constant Product Formula (x * y = k)**
   - The mathematical foundation of Uniswap-style AMMs
   - Price is determined by the ratio of reserves
   - Trades move along the bonding curve

2. **Price Impact / Slippage**
   - Larger trades relative to pool size have worse prices
   - Price impact grows non-linearly with trade size
   - Deep liquidity (high TVL) means less slippage

3. **Impermanent Loss**
   - The cost of providing liquidity vs holding
   - Caused by arbitrageurs rebalancing your position
   - Formula: IL = 2*sqrt(r)/(1+r) - 1
   - "Impermanent" because it reverses if price returns

4. **Arbitrage**
   - Keeps AMM prices aligned with external markets
   - Arbitrageurs profit from price discrepancies
   - Essential for price discovery

5. **Multi-hop Routing**
   - Trade tokens without direct pairs
   - Route through intermediate tokens (usually ETH/USDC)
   - Each hop adds fees and price impact

### Real-World Applications

- **Uniswap**: Largest DEX, pioneered constant product AMMs
- **Curve**: Optimized for stablecoin swaps (lower slippage)
- **Balancer**: Multi-token pools with custom weights
- **SushiSwap**: Uniswap fork with additional features

### LP Economics

- LPs earn trading fees (typically 0.3%)
- Must offset impermanent loss to be profitable
- High volume + stable prices = best LP returns
- Volatile pairs have high IL risk

### Further Reading

- [Uniswap V2 Whitepaper](https://uniswap.org/whitepaper.pdf)
- [Uniswap V3 Concentrated Liquidity](https://uniswap.org/whitepaper-v3.pdf)
- [Impermanent Loss Calculator](https://dailydefi.org/tools/impermanent-loss-calculator/)
- [Curve Stableswap Paper](https://curve.fi/files/stableswap-paper.pdf)